In [1]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
import numpy as np
import cvxpy as cp

# Problem statement

You’ve been invited to trade on the exchange of the north archipelago for one day only. 
The penguins have granted you access to their trusted news source: Iceberg. 
You’ll find all the information you need right there. Be aware that trading these foreign goods comes at a price. 
The more you trade in one good, the more expensive it will get. 
You will earn or lose money depending on how the market moves and what positions you hold. You have the opportunity to trade all sorts of new goods against yesterday's prices, just in time before the exchange opens for a new day.  You need to predict price moves based on the provided information.
This is the final stretch. Make it count! 

The initial available capital is 750000 SeaShells.

The Iceberg news sheet is available [here](https://small.fileditchstuff.me/s10/OdyMAIkSTCnDHdAnfpwJ.pdf).

Sample screenshot of the submission panel  
<img src="https://i.imgur.com/AwkoTHM.png" width="800" />

# Solution

The proportion $\pi_i$ (in $\%$) allocated to asset $i$ has to be an integer. The fee for this investment is $-90 \pi_i^2$.  
Note that $\pi_i$ can be negative, meaning that we take a short position in asset $i$.

To each product we associate a sentiment that indicates the direction and the magnitude of the expected change in the asset price, after the news break.  
Each sentiment is then mapped to a return value.

Note that these sentiments and returns are chosen at my discretion and are therefore debatable. However, they are needed to obtain a somewhat systematic solution to this problem, instead of a completely discretionary one.

In [2]:
sentiments = {
    'Refrigerators': '+',
    'Earrings': '++',
    'Blankets': '---',
    'Sleds': '--',
    'Sculptures': '++',
    'PS6': '+++',
    'Serum': '----',
    'Lamps': '+',
    'Chocolate': '-'
}

returns = {
    '+': 0.05,
    '++': 0.15,
    '+++': 0.25,
    '-': -0.05,
    '--': -0.1,
    '---': -0.4,
    '----': -0.6
}

products = list(sentiments.keys())

The portfolio optimization problem is
$$\max_{\pi_1,\ldots,\pi_9\in \mathbb Z} \quad \sum_{i=1}^9 7500r_i \pi_i  - 90 \pi_i^2 \quad \text{under the constraint}\quad  \sum_{i=1}^9 |\pi_i| \leq 100,$$
where $r_i$ denotes the return anticipated for asset $i$.

Without the integer constraints, the problem fits the framework of convex optimization.  
Below, we use `cvxpy` to obtain the solution without integer constraints.

In [3]:
rets = np.array([returns[sentiments[products[i-1]]] for i in range(1,10)])
pi = cp.Variable(9)
objective = cp.Minimize(90 * cp.sum_squares(pi) - 7500 * rets.T @ pi)
constraints = [cp.norm(pi, 1) <= 100]
prob = cp.Problem(objective, constraints)

prob.solve()
print('Optimal allocation without integer constraints:')
for i in range(9):
    print("Position in ", products[i], ': ', f"{pi.value[i]:,.2f}", '%', sep='')

Optimal allocation without integer constraints:
Position in Refrigerators: 2.08%
Position in Earrings: 6.25%
Position in Blankets: -16.67%
Position in Sleds: -4.17%
Position in Sculptures: 6.25%
Position in PS6: 10.42%
Position in Serum: -25.00%
Position in Lamps: 2.08%
Position in Chocolate: -2.08%


We use Mathematica below to numerically solve the fully constrained problem.

In [4]:
#building blocks for the Mathematica command
s1 = ' + '.join(['('+str(returns[sentiments[products[i-1]]])+')*p'+str(i)+'*7500-90*(p'+str(i)+')^2' for i in range(1,10)])
s2 = ' + '.join(['Abs[p'+str(i)+']' for i in range(1,10)]) + '<=100,'
s3 = ', '.join(['Element[p'+str(i)+', Integers]' for i in range(1,10)])
s4 = ', '.join(['p'+str(i) for i in range(1,10)])

In [5]:
#Mathematica command
'NMaximize[{'+s1+','+s2+s3+'}, {'+s4+'}]'

'NMaximize[{(0.05)*p1*7500-90*(p1)^2 + (0.15)*p2*7500-90*(p2)^2 + (-0.4)*p3*7500-90*(p3)^2 + (-0.1)*p4*7500-90*(p4)^2 + (0.15)*p5*7500-90*(p5)^2 + (0.25)*p6*7500-90*(p6)^2 + (-0.6)*p7*7500-90*(p7)^2 + (0.05)*p8*7500-90*(p8)^2 + (-0.05)*p9*7500-90*(p9)^2,Abs[p1] + Abs[p2] + Abs[p3] + Abs[p4] + Abs[p5] + Abs[p6] + Abs[p7] + Abs[p8] + Abs[p9]<=100,Element[p1, Integers], Element[p2, Integers], Element[p3, Integers], Element[p4, Integers], Element[p5, Integers], Element[p6, Integers], Element[p7, Integers], Element[p8, Integers], Element[p9, Integers]}, {p1, p2, p3, p4, p5, p6, p7, p8, p9}]'

In [6]:
session = WolframLanguageSession()
val_max, sol = session.evaluate(wlexpr('NMaximize[{'+s1+','+s2+s3+'}, {'+s4+'}]'))

Failed to converge to the requested accuracy or precision within 100 iterations.
Failed to converge to the requested accuracy or precision within 100 iterations.


In [7]:
print("Maximum profit achievable:", val_max)

Maximum profit achievable: 100740.0


In [8]:
print("Percentage of capital used: ", sum([abs(el[1]) for el in sol]), '%', sep='')

Percentage of capital used: 74%


In [9]:
for i in range(9):
    print("Position in ", products[i], ': ', sol[i][1], '%', sep='')

Position in Refrigerators: 2%
Position in Earrings: 6%
Position in Blankets: -17%
Position in Sleds: -4%
Position in Sculptures: 6%
Position in PS6: 10%
Position in Serum: -25%
Position in Lamps: 2%
Position in Chocolate: -2%


Values returned by Mathematica are consistent with those found using `cvxpy`.  
Convergence warnings are therefore not a problem.

# Results
Our picks performed better than expected, yielding a profit close to 141 000 SeaShells.

<img src="https://i.imgur.com/1I4WP0K.png" width="1000" />

Inspecting the json response from the site yields some more precise PNL data for each product, which we incorporate below.

In [10]:
pnls = {'Refrigerators': -49.29827880859375,
        'Earrings': 2325.331298828125,
        'Blankets': 15922.716796875,
        'Sleds': 7048.611328125,
        'Sculptures': 5596.8427734375,
        'PS6': 14216.5927734375,
        'Serum': 96703.71875,
        'Lamps': -359.08447265625,
        'Chocolate': -353.9310607910156}

From there we reverse engineer the true return for each product.

In [11]:
rets_true = {products[i]: (pnls[products[i]] + 90*sol[i][1]**2)/(7500*sol[i][1]) for i in range(9)}
print("True return for each product:")
rets_true

True return for each product:


{'Refrigerators': 0.020713448079427082,
 'Earrings': 0.12367402886284722,
 'Blankets': -0.32888405330882353,
 'Sleds': -0.2829537109375,
 'Sculptures': 0.19637428385416666,
 'PS6': 0.3095545703125,
 'Serum': -0.8157531666666666,
 'Lamps': 6.103515625e-05,
 'Chocolate': -0.000404595947265625}

For instance, the true return for serum was close to -81.6%.

As above we use Mathematica to find the optimal ex-post allocation.

In [12]:
s1 = ' + '.join(['('+'{:.20f}'.format(rets_true[products[i-1]])+')*p'+str(i)+'*7500-90*(p'+str(i)+')^2' for i in range(1,10)])
s2 = ' + '.join(['Abs[p'+str(i)+']' for i in range(1,10)]) + '<=100,'
s3 = ', '.join(['Element[p'+str(i)+', Integers]' for i in range(1,10)])
s4 = ', '.join(['p'+str(i) for i in range(1,10)])

session = WolframLanguageSession()
val_max, sol = session.evaluate(wlexpr('NMaximize[{'+s1+','+s2+s3+'}, {'+s4+'}]'))

Failed to converge to the requested accuracy or precision within 100 iterations.
Failed to converge to the requested accuracy or precision within 100 iterations.


In [13]:
print("Maximum profit achievable:", f"{val_max:.0f}")
print("Share of capital used: ", sum([abs(el[1]) for el in sol]), '%', sep='')
for i in range(9):
    print("Position in ", products[i], ': ', sol[i][1], '%', sep='')

Maximum profit achievable: 156823
Share of capital used: 87%
Position in Refrigerators: 1%
Position in Earrings: 5%
Position in Blankets: -14%
Position in Sleds: -12%
Position in Sculptures: 8%
Position in PS6: 13%
Position in Serum: -34%
Position in Lamps: 0%
Position in Chocolate: 0%
